# Data Download using Azure FarmBeats
Download of required satellite and weather data using Azure FarmBeats PaaS.

### Import 3rd party libraies

In [1]:
# Disable unnecessary logs 
import sys
import logging
logging.disable(sys.maxsize)

from datetime import datetime
import json
import numpy as np
import os
import pandas as pd
import rasterio
from azure.identity import ClientSecretCredential

### Import Farmbeats and Utilities

In [2]:
from azure.farmbeats.models import Farmer, Boundary, Polygon, SatelliteIngestionJobRequest, WeatherIngestionJobRequest
from azure.farmbeats import FarmBeatsClient
from utils.config import farmbeats_config
from utils.weather_util import WeatherUtil
from utils.satellite_util import SatelliteUtil
from utils.constants import CONSTANTS

### Farmbeats Configuration

In [3]:
# FarmBeats Client definition
credential = ClientSecretCredential(
    tenant_id="e21b7e4f-4b6c-4ead-bb26-b615da83f381",
    client_id="640027d7-3e96-49fc-8ecf-775f47a1e7b8",
    client_secret="rC.NHz~YPJn_xKGMh-OTg3M4P2kn6EbI90",
    authority="https://login.windows-ppe.net"
)

credential_scopes = ["https://farmbeats-dogfood.azure.net/.default"]

fb_client = FarmBeatsClient(
    base_url="https://agadhibjs-one.farmbeats-dogfood.azure.net",
    credential=credential,
    credential_scopes=credential_scopes,
    logging_enable=True
)

In [4]:
RUN = 103
NO_BOUNDARIES = 10

### Create Farmer

In [5]:
farmer_id = "annam_farmer"
try:
    farmer = fb_client.farmers.get(farmer_id=farmer_id)
    if farmer is not None:
        print("Farmer Exists")
    else:
        print("Farmer doesn't exist...Creating one ", end="", flush=True)
        farmer = fb_client.farmers.create_or_update(
            farmer_id=farmer_id,
            farmer=Farmer()
        )
except Exception as e:
    print(e)

Farmer Exists


### Create Boundaries

In [6]:
# Read 1000 farm geojsons from farms_1kmx1km.csv
locations_df = pd.read_csv("farms_sample_1kmx1km.csv")
locations_df["farms1"] = locations_df.farms.apply(json.loads)  # farm geojsons converted from string to list with numeric elements

boundaries = []

for i, item in enumerate(locations_df.farms1.values[:NO_BOUNDARIES]):
    boundary_id="boundary" + str(i) + str(RUN)
    try:
        boundary = fb_client.boundaries.get(
            farmer_id=farmer_id,
            boundary_id=boundary_id
        )
        if boundary is not None:
            print("Exist")
        else:
            print(f"Creating boundary with id {boundary_id}... ", end="")
            boundary = fb_client.boundaries.create_or_update(
                farmer_id=farmer_id,
                boundary_id=boundary_id,
                boundary=Boundary(
                    description="Created by SDK",
                    geometry=Polygon(
                        coordinates=[
                        item
                        ]
                    )
                )
            )
            print("Created")
    except Exception as e:
        print(e)
    boundaries.append(boundary)

Creating boundary with id boundary0103... Created
Creating boundary with id boundary1103... Created
Creating boundary with id boundary2103... Created
Creating boundary with id boundary3103... Created
Creating boundary with id boundary4103... Created
Creating boundary with id boundary5103... Created
Creating boundary with id boundary6103... Created
Creating boundary with id boundary7103... Created
Creating boundary with id boundary8103... Created
Creating boundary with id boundary9103... Created


###  Submit Satellite Jobs

In [7]:
# Start and End data for Satellite and Weather data to be pulled
start_dt = datetime.strptime(CONSTANTS["interp_date_start"], "%d-%m-%Y")
end_dt = datetime.strptime(CONSTANTS["interp_date_end"], "%d-%m-%Y")

satellite_jobs = []
for i, boundary in enumerate(boundaries[:NO_BOUNDARIES]):
    job_id = "satellitejob"+ str(i) + str(RUN)
    
    # Submit Satellite Job
    try:
        print("Queuing satellite job... ", end="", flush=True)
        satellite_job = fb_client.scenes.begin_create_satellite_data_ingestion_job(
            job_id=job_id,
            job=SatelliteIngestionJobRequest(
                farmer_id=boundary.farmer_id,
                boundary_id=boundary.id,
                start_date_time=start_dt,
                end_date_time=end_dt,
            ),
            polling=True
        )
        print("Submitted Satellite Job")

    except HttpResponseError as e:
        print(e)
        raise
    satellite_jobs.append(satellite_job)

Queuing satellite job... Submitted Satellite Job
Queuing satellite job... Submitted Satellite Job
Queuing satellite job... Submitted Satellite Job
Queuing satellite job... Submitted Satellite Job
Queuing satellite job... Submitted Satellite Job
Queuing satellite job... Submitted Satellite Job
Queuing satellite job... Submitted Satellite Job
Queuing satellite job... Submitted Satellite Job
Queuing satellite job... Submitted Satellite Job
Queuing satellite job... Submitted Satellite Job


### Check status of Satellite Jobs

In [8]:
for sat_job in satellite_jobs:
    print("Waiting")
    sat_job.result()

for sat_job in satellite_jobs:
    print(sat_job.result().as_dict()['id'])
    print(sat_job.status())
    
# TODO: Save job ids with Job request body to track failed jobs if any!

Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
satellitejob0103
Succeeded
satellitejob1103
Succeeded
satellitejob2103
Succeeded
satellitejob3103
Succeeded
satellitejob4103
Succeeded
satellitejob5103
Succeeded
satellitejob6103
Succeeded
satellitejob7103
Succeeded
satellitejob8103
Succeeded
satellitejob9103
Succeeded


### Submit Weather Jobs

In [9]:
# Weather API inputs
extension_id = "dtn.clearAg"
extension_api_name = "dailyhistorical"
extension_data_provider_api_key = farmbeats_config["weather_provider_key"]
extension_data_provider_app_id = farmbeats_config["weather_provider_id"]


In [10]:
weather_jobs = []
for i, boundary in enumerate(boundaries[:NO_BOUNDARIES]):
    job_id = "weatherjob"+ str(i) + str(RUN)
    st_unix = int(start_dt.timestamp())
    ed_unix = int(end_dt.timestamp())
    
    try:
        print("Queuing weather job... ", end="", flush=True)
        weather_job = fb_client.weather.begin_create_data_ingestion_job(
            job_id=job_id,
            job=WeatherIngestionJobRequest(
                farmer_id=boundary.farmer_id,
                boundary_id=boundary.id,
                extension_id= extension_id, 
                extension_api_name= extension_api_name, 
                extension_api_input= {"start": st_unix, "end": ed_unix},
                extension_data_provider_api_key= extension_data_provider_api_key,
                extension_data_provider_app_id=extension_data_provider_app_id
            ),
            polling=True
        )
        print("Submitted Weather Job")
    except HttpResponseError as e:
        print(e)
        raise
    weather_jobs.append(weather_job)

Queuing weather job... Submitted Weather Job
Queuing weather job... Submitted Weather Job
Queuing weather job... Submitted Weather Job
Queuing weather job... Submitted Weather Job
Queuing weather job... Submitted Weather Job
Queuing weather job... Submitted Weather Job
Queuing weather job... Submitted Weather Job
Queuing weather job... Submitted Weather Job
Queuing weather job... Submitted Weather Job
Queuing weather job... Submitted Weather Job


### Check status of Weather Jobs

In [11]:
for wth_job in weather_jobs:
    print("Waiting")
    wth_job.result()

for wth_job in weather_jobs:
    print(wth_job.result().as_dict()['id'])
    print(wth_job.status())
    
# TODO: Save job ids with Job request body to track failed jobs if any!

Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
weatherjob0103
Succeeded
weatherjob1103
Succeeded
weatherjob2103
Succeeded
weatherjob3103
Succeeded
weatherjob4103
Succeeded
weatherjob5103
Succeeded
weatherjob6103
Succeeded
weatherjob7103
Succeeded
weatherjob8103
Succeeded
weatherjob9103
Succeeded


### Download Satellite Data to Local

In [12]:
root_dir = "C:\\farmbeats\\"

In [14]:
df = SatelliteUtil(farmbeats_client = fb_client).download_and_get_sat_file_paths("annam-farmer", "b", 
                                                                              start_dt, 
                                                                              end_dt, 
                                                                              root_dir)
df.to_csv("satellite_paths.csv", index=None)

C:\azure-farmbeats-samples\evi_forecast\utils\satellite_util.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ).name.transform(len)


C:\azure-farmbeats-samples\evi_forecast\utils\satellite_util.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for x in df_allscenes_band.fileLink.values


### Download Weather Data to Local

In [17]:
for boundary in boundaries:
    weather_list = fb_client.weather.list(
            farmer_id=  boundary.farmer_id,
            boundary_id= boundary.id,
            extension_id="dtn.clearAg", 
            weather_data_type= "historical", 
            granularity="daily")

    weather_data = []
    for w_data in weather_list:
        weather_data.append(w_data)

    w_df = WeatherUtil.get_weather_data_df(weather_data)
    w_df.to_csv(boundary.id + "_weather_data.csv", index=False)

C:\azure-farmbeats-samples\evi_forecast\utils\weather_util.py:26: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_flat = pd.io.json.json_normalize([x.serialize() for x in weather_data])
